In [1]:
import os
from collections import Counter

import torch

In [2]:
MAX_LEN = 10
min_word_freq = 5


In [3]:
with open("/home/Ravikumar/Developer/chatbot/dataset/movie_conversations.txt","r") as c_file:
    conversations = c_file.readlines()

with open("/home/Ravikumar/Developer/chatbot/dataset/movie_lines.txt","r",encoding="iso-8859-1") as l_file:
    lines = l_file.readlines()

In [4]:
lines_dict = dict()
for line in lines:
    objects =  line.split(' +++$+++ ')
    lines_dict[objects[0]] = objects[-1]

In [9]:
import re
import unicodedata

def remove_punc(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct.lower()

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s


In [10]:
pairs = list()
for conv in conversations:
    ids = eval(conv.split("+++$+++")[-1])
    for i in range(len(ids)):
        qa_pairs = list()
        if i == len(ids)-1:
            break
        first = normalizeString(remove_punc(unicodeToAscii(lines_dict[ids[i]].strip())))
        second = remove_punc(lines_dict[ids[i+1]].strip())
        qa_pairs.append(first.split()[:MAX_LEN])
        qa_pairs.append(second.split()[:MAX_LEN])
        pairs.append(qa_pairs)
        
    # for i in range(len(ids)):
        # print(lines_dict[ids[i]])

In [11]:
word_freq = Counter()
for pair in pairs:
    word_freq.update(pair[0])
    word_freq.update(pair[1])

In [12]:
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k:v+1 for v,k in enumerate(words)}
word_map["<unk>"] = len(word_map)+1
word_map["<start>"] = len(word_map)+1
word_map["<end>"] = len(word_map)+1
word_map["<pad>"] = 0


In [13]:
print(f"Total words are : {len(word_map)}")

Total words are : 14117


In [14]:
import json
with open("/home/Ravikumar/Developer/chatbot/dataset/vocab.json","w") as v:
    json.dump(word_map,v)

In [20]:
def encode_question(words,word_map):
    enc = [word_map.get(word,word_map["<unk>"]) for word in words] +[word_map["<end>"]]+ [word_map["<pad>"]]* (MAX_LEN - len(words))
    return enc

In [21]:
def encode_reply(words,word_map):
    enc_c = [word_map["<start>"]] +  [word_map.get(word, word_map['<unk>']) for word in words] +[word_map["<pad>"]] * (MAX_LEN - len(words))
    return enc_c

In [22]:
pairs_encoded = []
for pair in pairs:
    qus = encode_question(pair[0], word_map)
    ans = encode_reply(pair[1], word_map)
    pairs_encoded.append((qus, ans))

In [23]:
pairs_encoded[0]

([1, 2, 3, 4, 5, 14114, 14114, 6, 7, 8, 14116],
 [14115, 9, 10, 11, 12, 13, 14, 14114, 15, 16, 17])

In [24]:
index2word = {v:k for k,v in word_map.items()}

In [29]:
print([index2word[idx] for idx in pairs_encoded[0][0]])
print([index2word[idx] for idx in pairs_encoded[0][1]])

['can', 'we', 'make', 'this', 'quick', '<unk>', '<unk>', 'and', 'andrew', 'barrett', '<end>']
['<start>', 'well', 'i', 'thought', 'wed', 'start', 'with', '<unk>', 'if', 'thats', 'okay']


In [30]:
with open('/home/Ravikumar/Developer/chatbot/dataset/pairs_encoded.json', 'w') as p:
    json.dump(pairs_encoded, p)

In [31]:
print("done")

done


In [32]:
print("Sequence size of input(question) :",len(pairs_encoded[100][0]))
print("Sequence size of output(reply)   :",len(pairs_encoded[100][1]))

Sequence size of input(question) : 11
Sequence size of output(reply)   : 11


In [33]:
print("Total number of samples : ",len(pairs_encoded))

Total number of samples :  221616


In [34]:
def masking(seq):
    mask = list()
    for s in seq:
        if s == 0:
            mask.append(False)
        else:
            mask.append(True)
    return torch.tensor(mask)

In [35]:
# 25th example to show the mask and input sequence 
ins = pairs_encoded[25][0]
outs = pairs_encoded[25][1]

print(f"input sequence : {ins}  #input_size : {len(ins)}")
print(f"output sequence : {outs} #input_size : {len(outs)}")
print("Mask : ",masking(outs))

input sequence : [110, 92, 161, 14116, 0, 0, 0, 0, 0, 0, 0]  #input_size : 11
output sequence : [14115, 10, 162, 2, 163, 164, 165, 14114, 0, 0, 0] #input_size : 11
Mask :  tensor([ True,  True,  True,  True,  True,  True,  True,  True, False, False,
        False])


14117